In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

data_dir = Path("../data/Cleaned Data")

understat_files = sorted(data_dir.glob("*understat*.csv"))
prem_files = sorted(data_dir.glob("*prem*.csv"))

if not understat_files:
    raise FileNotFoundError("No cleaned UNDERSTAT CSVs found")

df_under = pd.concat((pd.read_csv(f, parse_dates=["date"]) for f in understat_files), ignore_index=True)
df_prem = pd.concat((pd.read_csv(f, parse_dates=["date"]) for f in prem_files), ignore_index=True) if prem_files else pd.DataFrame()

print(f"UNDERSTAT rows: {len(df_under)}, PREM rows: {len(df_prem)}")

def scatter_with_regression(df, xcol, ycol, title, xlabel=None, ylabel=None):
    """Scatter plot with regression line"""
    tmp = df[[xcol, ycol]].dropna().copy()
    tmp[xcol] = pd.to_numeric(tmp[xcol], errors="coerce")
    tmp[ycol] = pd.to_numeric(tmp[ycol], errors="coerce")
    tmp = tmp.dropna()
    if tmp.empty:
        print(f"Skipping {title}: no numeric data")
        return
    x = tmp[xcol].values
    y = tmp[ycol].values
    coeff = np.polyfit(x, y, 1)
    x_sorted = np.linspace(x.min(), x.max(), 200)
    y_fit = np.polyval(coeff, x_sorted)
    
    fig = px.scatter(tmp, x=xcol, y=ycol, title=title, labels={xcol: xlabel or xcol, ycol: ylabel or ycol})
    fig.add_trace(go.Scatter(x=x_sorted, y=y_fit, mode="lines", name=f"fit: y={coeff[0]:.3f}x+{coeff[1]:.3f}", line=dict(color="red")))
    fig.show()

# Graph 1 – xG Difference vs Match Points (scatter + regression line)
if {"xG","xGA","pts"}.issubset(df_under.columns):
    df_under["xG_diff"] = pd.to_numeric(df_under["xG"], errors="coerce") - pd.to_numeric(df_under["xGA"], errors="coerce")
    scatter_with_regression(df_under, "xG_diff", "pts", "xG Difference vs Match Points", "xG Difference", "Match Points")

# Graph 2 – Rolling Form vs Next Match Points (scatter + regression line)
if "club_name" in df_under.columns and "pts" in df_under.columns and "date" in df_under.columns:
    df_under = df_under.sort_values(["club_name","date"])
    df_under["form_3"] = df_under.groupby("club_name")["pts"].transform(lambda x: x.rolling(3, min_periods=1).mean().shift(1))
    scatter_with_regression(df_under, "form_3", "pts", "Rolling Form (3 matches) vs Next Match Points", "Form (3-match avg)", "Next Match Points")

# Graph 3 – xG vs Expected Points (continuous outcome)
if {"xG","xpts"}.issubset(df_under.columns):
    scatter_with_regression(df_under, "xG", "xpts", "Expected Goals vs Expected Points", "xG", "Expected Points (xpts)")

print("✓ Graphs generated successfully!")

UNDERSTAT rows: 3158, PREM rows: 0


✓ Graphs generated successfully!
